In [17]:
# import all libraries
import json
import cv2
import os
import numpy as np
from pycocotools import mask as maskUtils
from PIL import Image
import matplotlib.pyplot as plt

## Task1- Convert VGG Via Annotation file into Yolo Format

In [5]:
# Set the class mapping dictionary
class_mapping = {"dog": 0, "cat": 1}

In [6]:
# Set the output file paths
json_file_path = "cat_dog_annotations.json"
output_file_path = "output.txt"

In [7]:
# Load the JSON file
with open(json_file_path, "r") as f:
    data = json.load(f)

In [8]:
# Loop over the images in the JSON
for image_id, (image_filename, image_data) in enumerate(data["_via_img_metadata"].items()):
    # Get the image width and height
    #default_height=[]
    #default_width=[]
    width = image_data.get("width", 400)
    height = image_data.get("height", 700)

    # Open the output file for this image
    output_filename = f"{image_id}.txt"
    with open(output_filename, "w") as output_file:
        # Loop over the regions in the image
        for region in image_data["regions"]:
            # Get the class name and convert it to an integer
            class_name = list(region["region_attributes"]["class"].keys())[0]
            class_id = class_mapping[class_name]

            # Get the bounding box coordinates and convert them to YOLO format
            x = region["shape_attributes"]["x"]
            y = region["shape_attributes"]["y"]
            w = region["shape_attributes"]["width"]
            h = region["shape_attributes"]["height"]
            x_center = x + w / 2
            y_center = y + h / 2
            x_center /= width
            y_center /= height
            w /= width
            h /= height

            # Write the object detection annotation to the output file
            output_file.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

## Task2- Convert VGG Via Annotation file into COCO Format

In [9]:
# Initialize the COCO dataset object
dataset = {
    "info": {},
    "licenses": [],
    "categories": [{"id": 1, "name": "dog"}, {"id": 2, "name": "cat"}],
    "images": [],
    "annotations": []
}

In [10]:
# Add images and annotations to the dataset object
image_id = 1
annotation_id = 1
for filename in data['_via_img_metadata']:
    # Add the image to the dataset
    img = Image.open("1.jpeg")
    width, height = img.size
    dataset['images'].append({
        "id": image_id,
        "width": width,
        "height": height,
        "file_name": filename
    })

    # Add annotations to the dataset
    for region in data['_via_img_metadata'][filename]['regions']:
        x, y, w, h = region['shape_attributes']['x'], region['shape_attributes']['y'], region['shape_attributes']['width'], region['shape_attributes']['height']
        category = region['region_attributes']['class']
        if category == {'dog': True}:
            category_id = 1
        elif category == {'cat': True}:
            category_id = 2
        else:
            continue
        area = w * h
        bbox = [x, y, w, h]
        segmentation = [[x, y, x+w, y, x+w, y+h, x, y+h]]
        annotation = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,
            "bbox": bbox,
            "area": area,
            "segmentation": segmentation,
            "iscrowd": 0
        }
        dataset['annotations'].append(annotation)
        annotation_id += 1

    image_id += 1

# Save the dataset object to a JSON file
with open('coco_annotations.json', 'w') as f:
    json.dump(dataset, f)

## Task3- Convert VGG Via Annotation file into YOLO Format plus augment images using noise and changing brightness

In [11]:
import json

# Define the class label (0 since there are no objects to detect)
class_label = 0

# Load the JSON file
with open('brightness_noise.json', 'r') as f:
    data = json.load(f)

# Loop through each image and generate the YOLO format string
for img_data in data.values():
    # Extract the image filename and size
    filename = img_data['filename']
    width = img_data['size']
    height = img_data['size']

    # Generate the YOLO format string
    yolo_str = f"{class_label} {filename} {width} {height}"
    
    # Print the YOLO format string
    print(yolo_str)

0 1_0.jpeg 34670 34670
0 1_1.jpeg 33102 33102
0 1_2.jpeg 29556 29556
0 1_3.jpeg 30750 30750
0 1_4.jpeg 36035 36035
0 2_0.jpeg 4735 4735
0 2_1.jpeg 4727 4727
0 2_2.jpeg 5437 5437
0 2_3.jpeg 4650 4650
0 2_4.jpeg 5069 5069
0 3_0.jpeg 9559 9559
0 3_1.jpeg 9728 9728
0 3_2.jpeg 9638 9638
0 3_3.jpeg 9550 9550
0 3_4.jpeg 9663 9663
0 4_0.jpeg 167572 167572
0 4_1.jpeg 164788 164788
0 4_2.jpeg 152478 152478
0 4_3.jpeg 155606 155606
0 4_4.jpeg 151526 151526
0 5_0.jpeg 46602 46602
0 5_1.jpeg 49043 49043
0 5_2.jpeg 50305 50305
0 5_3.jpeg 46988 46988
0 5_4.jpeg 49375 49375
0 6_0.jpeg 14328 14328
0 6_1.jpeg 14312 14312
0 6_2.jpeg 17076 17076
0 6_3.jpeg 14957 14957
0 6_4.jpeg 14322 14322
0 7_0.jpeg 26892 26892
0 7_1.jpeg 24724 24724
0 7_2.jpeg 26059 26059
0 7_3.jpeg 26240 26240
0 7_4.jpeg 26868 26868
0 1_0.jpeg 88773 88773
0 1_1.jpeg 88818 88818
0 1_2.jpeg 88904 88904
0 1_3.jpeg 88866 88866
0 1_4.jpeg 88995 88995
0 2_0.jpeg 11255 11255
0 2_1.jpeg 11282 11282
0 2_2.jpeg 11266 11266
0 2_3.jpeg 11253 1125

## Task4- Convert VGG Via Annotation file into YOLO Format plus augment images for resizing

In [ ]:
# define the path to the image directory
img_dir = "resize"

In [ ]:
# loop through all the images in the directory
for filename in os.listdir(img_dir):
    if filename.endswith(".jpeg"):

        # open the image file
        img = Image.open(os.path.join(img_dir, filename))

        # get the original image size
        original_size = img.size

        # resize the image to 0.5 times the original size
        new_size = tuple(int(x*0.5) for x in original_size)
        img_0_5 = img.resize(new_size)

        # resize the image to 2 times the original size
        new_size = tuple(int(x*2) for x in original_size)
        img_2 = img.resize(new_size)

        # resize the image to 3 times the original size
        new_size = tuple(int(x*3) for x in original_size)
        img_3 = img.resize(new_size)

        # save the augmented images
        img.save(os.path.join(img_dir, filename))
        img_0_5.save(os.path.join(img_dir, filename[:-5] + "_0.5size.jpeg"))
        img_2.save(os.path.join(img_dir, filename[:-5] + "_2size.jpeg"))
        img_3.save(os.path.join(img_dir, filename[:-5] + "_3size.jpeg"))


In [21]:
# Define the class label (0 since there are no objects to detect)
class_label = 0

In [22]:
# Load the JSON file
with open("resize.json", 'r') as f:
    data = json.load(f)

In [23]:
# Loop through each image and generate the YOLO format string
for img_data in data.values():
    # Extract the image filename and size
    filename = img_data['filename']
    width = img_data['size']
    height = img_data['size']

    # Generate the YOLO format string
    yolo_str = f"{class_label} {filename} {width} {height}"
    
    # Print the YOLO format string
    print(yolo_str)

0 1.jpeg 43643 43643
0 1_0.5size.jpeg 15645 15645
0 1_2size.jpeg 115623 115623
0 1_3size.jpeg 217909 217909
0 2.jpeg 5977 5977
0 2_0.5size.jpeg 2572 2572
0 2_2size.jpeg 14195 14195
0 2_3size.jpeg 26421 26421
0 3.jpeg 8711 8711
0 3_0.5size.jpeg 3137 3137
0 3_2size.jpeg 21368 21368
0 3_3size.jpeg 38729 38729
0 4.jpeg 156298 156298
0 4_0.5size.jpeg 45334 45334
0 4_2size.jpeg 359339 359339
0 4_3size.jpeg 660098 660098
0 5.jpeg 51205 51205
0 5_0.5size.jpeg 16510 16510
0 5_2size.jpeg 119403 119403
0 5_3size.jpeg 221181 221181
0 6.jpeg 18089 18089
0 6_0.5size.jpeg 6154 6154
0 6_2size.jpeg 47548 47548
0 6_3size.jpeg 86399 86399
0 7.jpeg 46018 46018
0 7_0.5size.jpeg 15772 15772
0 7_2size.jpeg 125045 125045
0 7_3size.jpeg 230166 230166
